In [1]:
## main_1: load result

import pickle as pk
import numpy as np
import csv
num_trials = 10 
num_envs = 4

In [2]:
d_result = {}
list_d_result = []
for i in range(num_trials):
    for file_no in [0, 1, 2, 3]: ## change_2: env 0 ==> file 0; env 1,2,3 ==> file 3, env 4 ==> file 4, env 5 ==> file 5
        with open('datasets/' + 'd_result_' + str(i) + '_' + str(file_no) + '.pickle', 'rb') as handle:
            d = pk.load(handle)
            list_d_result.append(d)
            d_result.update(d)

In [3]:
len(list(d_result.keys())) ## trials 10 X envs 4 X model 3 X 5 algs == 9600

600

In [4]:
# d_result
for it in range(10):
    for ie in range(4):
        print(it, ie, d_result[str(it), str(ie), str('Siamese'), str('ridge')])

0 0 0.42058566
0 1 0.41200298
0 2 0.41412732
0 3 0.4930225
1 0 0.29541332
1 1 0.26415792
1 2 0.33864123
1 3 0.2907431
2 0 0.19757026
2 1 0.22277905
2 2 0.26399532
2 3 0.32162443
3 0 0.35010374
3 1 0.3908381
3 2 0.48901534
3 3 0.47869858
4 0 0.41603473
4 1 0.4489188
4 2 0.46232086
4 3 0.49285206
5 0 0.40129834
5 1 0.39106917
5 2 0.29580173
5 3 0.36554182
6 0 0.27426094
6 1 0.25008294
6 2 0.2961534
6 3 0.40946215
7 0 0.22205842
7 1 0.31284878
7 2 0.32962677
7 3 0.33557275
8 0 0.30527502
8 1 0.288986
8 2 0.36516845
8 3 0.35367393
9 0 0.3318836
9 1 0.4571868
9 2 0.39556617
9 3 0.43208802


In [5]:
d_env_no_vs_env_name = {
    0: 'PBE-D3',
    1: 'BEEF-vdw',
    2: 'RPBE',
    3: 'scan-rVV10',
}
list_line = [['env_no (env_name)', 'alg', 'model_name', 'mean ± std']]
d_line = {}
for env_no in [0,1,2,3]: ## change_2:
    d_line[env_no] = {}
    for alg in ['ridge','elastic','krr','svr']:
        d_line[env_no][alg] = {}
        list_model_name = ['regular', 'PCA', 'Siamese'] 
        # for _nhn in [128, 64, 32, 16, 8]:
        #     for _lr in [1e-4, 1e-3, 1e-2]:
        #         for _sc in [0, 1]:
#         for _nhn in [128]:
#             for _lr in [1e-2, 1e-3]:
#                 for _sc in [0]:
        
#                     mn = 'Siamese_' + str(_nhn) + '_' + str(_lr) + '_' + str(_sc)
#                     list_model_name.append(mn)
        for model_name in list_model_name:
            list_result = []
            for trial_no in range(num_trials):
                key = (str(trial_no), str(env_no), str(model_name), str(alg))
                result = d_result[key]
                list_result.append(result)
            mean, std = np.mean(list_result), np.std(list_result)
            _mean_std = '{:.2f} ± {:.2f}'.format(mean, std)
            line = [str(env_no) + ' (' + str(d_env_no_vs_env_name[env_no]) + ')', alg, model_name, _mean_std]
            list_line.append(line)
            
            if model_name == 'regular':
                d_line[env_no][alg]['regular'] = (mean, std)
            elif model_name == 'PCA':
                d_line[env_no][alg]['PCA'] = (mean, std)
            else:
                if 'Siamese' not in d_line[env_no][alg].keys():
                    d_line[env_no][alg]['Siamese'] = (mean, std)  
                else:
                    old_mean = d_line[env_no][alg]['Siamese'][0]
                    if mean < old_mean:
                        d_line[env_no][alg]['Siamese'] = (mean, std) ## update
                    else:
                        pass ## do not update

In [6]:
list_row = []
for env_no in [0,1,2,3]: ## change_2:

    list_row.append(['env no: ' + str(env_no), '', '', ''])
    list_row.append(['alg\model', 'regular', 'PCA', 'Siamese'])
    
    for alg in ['ridge','elastic','krr','svr']:
        row = [alg]
        for model_name in ['regular', 'PCA', 'Siamese']:
            mean, std = d_line[env_no][alg][model_name]
            _mean_std = '{:.2f} ± {:.2f}'.format(mean, std)
            row.append(_mean_std)
        list_row.append(row)  
    list_row.append(['', '', '', ''])
    list_row.append(['', '', '', ''])


In [7]:
filename = "datasets/result_128_1_v3.csv"
    
with open(filename, 'w', newline='', encoding='utf-8') as csvfile: 
    csvwriter = csv.writer(csvfile) 
        
    # # writing the fields 
    # csvwriter.writerow(fields)         
    # writing the data rows 
    csvwriter.writerows(list_row)